校验kmeans的结果，合成信号值， 多头信号1 数据应该集中数据头部，  空头信号-1 数据应该集中在数据尾部  中间处于不动即0

In [1]:
import os
import pandas as pd
from lumina.genetic.fusion import Rotor
from lumina.genetic import Actuator

In [4]:
code = 'im'
def fetch_rotor(name):
    base_path = '/workspace/worker/pj/Chrono/lumina/abily/records/aicso2/ims/kmeans'
    rotor = Rotor.from_pickle(
            path=os.path.join(base_path, code.lower()), name=name)
    return rotor

from kdutils.file import fetch_file_data
base_path = '/workspace/worker/pj/Chrono/lumina/abily/records/'
#factors_data = fetch_file_data(base_path=base_path,
#                          method='aicso2',
#                          g_instruments='ims',
#                          datasets=['train_data', 'val_data'])

def fetch_data():
    base_path = '/workspace/worker/pj/Chrono/lumina/abily/records/'
    filename = os.path.join(base_path, 'aicso2', 'ims', 'merge',"val_data.feather")
    factors_data = pd.read_feather(filename).sort_values(
        by=['trade_time', 'code'])
    return factors_data

def create_postions(k_split, filter_strategies, total_data):
    actuator = Actuator(k_split=k_split)

    strategies_data = actuator.calculate(strategies_infos=filter_strategies,
                                         total_data=total_data)
    return strategies_data


def merge_signals(strategies_data, filter_strategies):
    actuator = Actuator(k_split=4)
    weights_data = actuator.fitness_weight(strategies_infos=filter_strategies)
    positions_data = actuator.fitness_signal(
        strategies_infos=filter_strategies,
        strategies_data=strategies_data,
        weights_data=weights_data)
    return positions_data, weights_data

total_data = fetch_file_data(base_path=base_path,
                          method='aicso2',
                          g_instruments='ims',
                          datasets=['train_data', 'val_data'])
total_data['trade_time'] = pd.to_datetime(total_data['trade_time'])
basic_rotor = fetch_rotor('1046921830')

strategies_data = create_postions(k_split=4,
                                      filter_strategies=basic_rotor.strategies,
                                      total_data=total_data)
positions_data, weights_data = merge_signals(strategies_data,
                                                 basic_rotor.strategies)
positions_data.name = 'value'
positions_data = positions_data.reset_index().set_index('trade_time')
positions_data.head()

predict groups model::40.0%

/workspace/worker/env/lingua/lib/python3.8/site-packages/Finance_Ultron-1.7.8-py3.8-linux-x86_64.egg/ultron/factor/genetic/geneticist/operators.py:320: RuntimeWarning: divide by zero encountered in true_divide
  return eval(expression).transform(total_data,


predict groups model::88.0%%

/workspace/worker/env/lingua/lib/python3.8/site-packages/Finance_Ultron-1.7.8-py3.8-linux-x86_64.egg/ultron/factor/genetic/geneticist/operators.py:320: RuntimeWarning: divide by zero encountered in true_divide
  return eval(expression).transform(total_data,


predict groups model::100.0%

/workspace/worker/env/lingua/lib/python3.8/site-packages/Finance_Ultron-1.7.8-py3.8-linux-x86_64.egg/ultron/factor/genetic/geneticist/operators.py:320: RuntimeWarning: divide by zero encountered in true_divide
  return eval(expression).transform(total_data,


predict groups model::100.0%

,code,value
trade_time,,
2022-07-25 09:30:00,IM,0.0
2022-07-25 09:31:00,IM,0.0
2022-07-25 09:32:00,IM,0.0
2022-07-25 09:33:00,IM,0.0
2022-07-25 09:34:00,IM,0.0


In [5]:
roto1 = fetch_rotor('1046921830')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:-0.15528037532185335, long:0.6273799464654383, zero:0.08150273022077939, mapping:{0: -1, 1: 1, 2: 0}


In [6]:
roto1 = fetch_rotor('1050954659')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:0.08150273022077939, long:0.6273799464654383, zero:-0.15528037532185335, mapping:{0: 0, 1: 1, 2: -1}


In [7]:
roto1 = fetch_rotor('1051075300')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:0.6273799464654383, long:0.08150273022077939, zero:-0.15528037532185335, mapping:{0: 0, 1: -1, 2: 1}


In [8]:
roto1 = fetch_rotor('1058854373')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:0.08150273022077939, long:-0.15528037532185335, zero:0.6273799464654383, mapping:{0: 1, 1: 0, 2: -1}


In [9]:
roto1 = fetch_rotor('1081998918')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:0.6273799464654383, long:-0.15528037532185335, zero:0.08150273022077939, mapping:{0: 1, 1: -1, 2: 0}


In [10]:
roto1 = fetch_rotor('1093058596')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:-0.15528037532185335, long:0.08150273022077939, zero:0.6273799464654383, mapping:{0: -1, 1: 0, 2: 1}


In [16]:
#mapping:{0: -1, 1: 1, 2: 0}

import joblib

infos = joblib.load('/workspace/worker/pj/Chrono/lumina/abily/records/aicso2/ims/metrics/3_-1_1_0.pkl')

In [17]:
infos.empyrical

EmpyricalTuple(name='3_-1_1_0', annual_return=1.3045637939889945, annual_volatility=0.24710814809378184, calmar=12.271194116941496, sharpe=3.5053106053793575, max_drawdown=-0.10631107140485423, sortino=7.269827168194353, returns_series=trade_time
2022-07-25    0.0115
2022-07-26    0.0143
2022-07-27    0.0211
2022-07-28   -0.0024
2022-07-29   -0.0020
               ...  
2024-11-29    0.0153
2024-12-02    0.0160
2024-12-03   -0.0156
2024-12-04   -0.0113
2024-12-05    0.0003
Name: ret, Length: 575, dtype: float64)